In [1]:
import os
import pandas as pd
import numpy as np

In [ ]:
# https://www.arabidopsis.org/download/file?path=Public_Data_Releases/TAIR_Data_20230930/TAIR_Data_20230930/Araport11_GTF_genes_transposons.Oct2023.gtf.gz
# https://www.arabidopsis.org/download/file?path=Genes/Araport11_genome_release/Araport11_GTF_genes_transposons.current.gtf.gz

# 6 MBOct 7, 2024
! wget https://www.arabidopsis.org/download/file?path=Genes/TAIR10_genome_release/TAIR10_chromosome_files/TAIR10_chr_all.fas.gz \
-O ../ref/TAIR10_chr_all.fas.gz
! gunzip ../ref/TAIR10_chr_all.fas.gz

! wget https://www.arabidopsis.org/api/download-files/download?filePath=Genes/Araport11_genome_release/Araport11_GTF_genes_transposons.current.gtf.gz \
-O ../ref/tair10/Araport11_GTF_genes_transposons.current.gtf.gz

! gtfToGenePred -genePredExt -ignoreGroupsWithoutExons ../ref/tair10/Araport11_GTF_genes_transposons.current.gtf.gz stdout \
| genePredToBed stdin ../ref/tair10/Araport11_GTF_genes_transposons.current.bed

! awk '$7!=$8' ../ref/tair10/Araport11_GTF_genes_transposons.current.bed > ../ref/tair10/Araport11_GTF_genes_transposons.current.pc.bed

! zcat ../ref/tair10/Araport11_GTF_genes_transposons.current.gtf.gz \
| grep -i 'uorf' | sed 's/uORF/exon/' \
| gtfToGenePred -genePredExt -ignoreGroupsWithoutExons stdin stdout \
| awk 'BEGIN{OFS="\t"} {print $1,$2,$3,$4,$5,$4,$5,$8,$9,$10,$11,$12,$13,$14,$15}' \
> ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.gp

! bedtools getfasta -bed ../ref/tair10/Araport11_GTF_genes_transposons.current.pc.bed -fi ../ref/TAIR10_chr_all.fas \
-s -split -name -fo ../ref/tair10/Araport11_GTF_genes_transposons.current.pc.fa

##### Remove uORFs that have no AUG start codons

In [2]:
! genePredToBed ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.gp ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.bed

! awk 'BEGIN{OFS="\t"} {print $1,$7,$8,$4,$5,$6}' ../ref/tair10/Araport11_GTF_genes_transposons.current.pc.bed \
| bedtools intersect -a ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.bed -b - -s -split -wao \
| awk '$NF==0' | cut -f-12 > ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.uniq.bed

! wc -l ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.bed

170 ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.bed


In [3]:
! wc -l ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.uniq.bed

166 ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.uniq.bed


In [4]:
! bedtools getfasta -bed ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.uniq.bed -fi ../ref/tair10/TAIR10_chr_all.fas -split -name -s -tab \
| cut -f2 | cut -c-3 \
| paste - ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.uniq.bed \
| awk '/^ATG/' | cut -f2- \
> ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.uniq.bed2

In [5]:
! bedtools getfasta -bed ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.uniq.bed2 -fi ../ref/tair10/TAIR10_chr_all.fas -split -name -s -tab \
| cut -f2 | rev | cut -c-3 \
| paste - ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.uniq.bed2 \
| awk '!/^GAC/' | cut -f2- \
> ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.uniq.bed

In [6]:
! wc -l ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.uniq.bed

161 ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.uniq.bed


#### Compare ORFs found in RIBOSS orf_finder and ribotricer index

In [7]:
rt_candidate_orfs = pd.read_csv('../ref/tair10/TAIR10_chr_all_candidate_orfs.tsv', sep='\t')
# rt_candidate_orfs = rt_candidate_orfs[rt_candidate_orfs.transcript_type.str.contains('protein_coding')].copy()

df = pd.read_pickle('../ref/tair10/Araport11_GTF_genes_transposons.current.orf_finder.pkl.gz')
dfp = df[df['Strand']=='+'].copy()
dfm = df[df['Strand']=='-'].copy()
dfp['ORF_ID'] = dfp.tid + '_' + (dfp.Start + 1).astype(int).astype(str) + '_' + (dfp.End - 3).astype(int).astype(str) + '_' + (dfp.ORF_length - 3).astype(int).astype(str)
dfm['ORF_ID'] = dfm.tid + '_' + (dfm.Start + 4).astype(int).astype(str) + '_' + (dfm.End).astype(int).astype(str) + '_' + (dfm.ORF_length - 3).astype(int).astype(str)

df = pd.concat([dfp,dfm])
df_rt = pd.merge(df,rt_candidate_orfs, on='ORF_ID', how='outer')

In [8]:
rt_candidate_orfs[['tid','Start','End','ORF_length']] = rt_candidate_orfs.ORF_ID.str.split('_', expand=True)
rt_candidate_orfs[['chrom','Start','End','tid','ORF_length','strand']].to_csv('../ref/tair10/TAIR10_chr_all_candidate_orfs.bed', header=None, index=None, sep='\t')

In [9]:
df_rt.dropna().shape, df_rt.shape, df.shape, df[df.ORF_type!='mORF'].shape, rt_candidate_orfs.shape, rt_candidate_orfs[rt_candidate_orfs.ORF_type!='annotated'].shape

((109339, 22),
 (634108, 22),
 (455905, 12),
 (407757, 12),
 (287542, 15),
 (239353, 15))

109339 are in common between RIBOSS orf_finder and ribotricer index

In [10]:
df_rt.dropna(inplace=True)
df_rt['oid'] = df_rt.tid + '__' + df_rt.ORF_start.astype(int).astype(str) + '-' + df_rt.ORF_end.astype(int).astype(str)
df_rt[['oid','ORF_ID']].to_csv('../ref/tair10/orf_finder_ribotricer.intersection.txt', index=None, sep='\t')

#### Ribo-Seq ORFs found by RIBOSS orf_finder and ribotricer index

In [11]:
df = pd.read_pickle('../ref/tair10/Araport11_GTF_genes_transposons.current.orf_finder.pkl.gz')
df['Start'] = df.Start.astype(int)
df['End'] = df.End.astype(int)

df.drop('ORF_range', axis=1, inplace=True)
df.to_csv('../results/wu_2024_datasets/orf_finder/Araport11_GTF_genes_transposons.current.orf_finder.txt', index=None, sep='\t')

In [12]:
! awk 'NR>1 && !/annotated/ && gsub(/_/, "\t", $1) {print $8,$1,$9,$2}' ../ref/tair10/TAIR10_chr_all_candidate_orfs.tsv \
| awk 'BEGIN{OFS="\t"}{print $1,$3,$4,$2 "|" $7,$5,$6}' \
| bedtools intersect -a - -b ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.uniq.bed -s -split -wo \
| awk '(($2==$8) || ($3==$9)) || (($2-1)==$8) || ($3==$9)' \
> ../ref/tair10/Araport11_GTF_genes_transposons.ribotricer.txt

! awk 'BEGIN{OFS="\t"} NR>1 {print $1,$8,$9,$2 "__" $5 "-" $6 "|" $NF,$7,$3}' ../results/wu_2024_datasets/orf_finder/Araport11_GTF_genes_transposons.current.orf_finder.txt \
| bedtools intersect -a - -b ../ref/tair10/Araport11_GTF_genes_transposons.current.uorfs.uniq.bed -s -split -wo \
| awk '(($2==$8) || ($3==$9))' \
> ../ref/tair10/Araport11_GTF_genes_transposons.orf_finder.txt

! comm -12 \
<(cut -f10 ../ref/tair10/Araport11_GTF_genes_transposons.ribotricer.txt | sort -u) \
<(cut -f10 ../ref/tair10/Araport11_GTF_genes_transposons.orf_finder.txt | sort -u) \
> ../ref/tair10/Araport11_GTF_genes_transposons.ribotricer.intersection.txt

! comm -23 \
<(cut -f10 ../ref/tair10/Araport11_GTF_genes_transposons.ribotricer.txt | sort -u) \
<(cut -f10 ../ref/tair10/Araport11_GTF_genes_transposons.orf_finder.txt | sort -u) \
> ../ref/tair10/Araport11_GTF_genes_transposons.ribotricer.uniq.txt

! comm -13 \
<(cut -f10 ../ref/tair10/Araport11_GTF_genes_transposons.ribotricer.txt | sort -u) \
<(cut -f10 ../ref/tair10/Araport11_GTF_genes_transposons.orf_finder.txt | sort -u) \
> ../ref/tair10/Araport11_GTF_genes_transposons.orf_finder.uniq.txt

In [13]:
! wc -l ../ref/tair10/Araport11_GTF_genes_transposons.orf_finder.txt

150 ../ref/tair10/Araport11_GTF_genes_transposons.orf_finder.txt


In [14]:
rs_of = pd.read_csv('../ref/tair10/Araport11_GTF_genes_transposons.orf_finder.txt', sep='\t', header=None)
rs_of['ORF_length'] = rs_of[16].apply(lambda x: np.sum([int(i) for i in x.split(',')[:-1]]))
rs_of = rs_of[rs_of[4]==rs_of.ORF_length].copy()
rs_of['oid'] = rs_of[3].str.split('|').str[0]

rs_of.rename(columns={9:'Araport_uORF'}, inplace=True)
rs_of.shape
rs_of.drop_duplicates('Araport_uORF').shape

(86, 21)

In [15]:
rs_rt = pd.read_csv('../ref/tair10/Araport11_GTF_genes_transposons.ribotricer.txt', sep='\t', header=None)
rs_rt['ORF_ID'] = rs_rt[3].str.split('|').str[0] + '_' + rs_rt[1].astype(str) + '_' + rs_rt[2].astype(str) + '_' + rs_rt[4].astype(str)
rs_rt.rename(columns={9:'Araport_uORF'}, inplace=True)

rs_rt['ORF_length'] = rs_rt[16].apply(lambda x: np.sum([int(i) for i in x.split(',')[:-1]]))
rs_rt = rs_rt[(rs_rt[4]+3)==rs_rt.ORF_length].copy()
rs_rt.drop_duplicates('Araport_uORF').shape

(81, 21)

##### Get a list of Ribo-Seq ORF in common

In [16]:
set_rs_rt = set(rs_rt['Araport_uORF'].tolist())
set_rs_of = set(rs_of['Araport_uORF'].tolist())

len(set_rs_rt.intersection(set_rs_of))

78

In [17]:
# len(set_rs_rt.union(set_rs_of))
pd.DataFrame({'Araport_uORF':list(set_rs_rt.intersection(set_rs_of))}).to_csv('../ref/tair10/Araport_uORF.intersection.txt', index=None)